### Initialize ellipsoids with DPD

In [1]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
import freud
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        density,
        bond_L=0.1, #T-T bond length
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.density = density
        N=lengths*num_mols
        L = np.cbrt(N/ self.density)
        self.L = L
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        rand_range = ((self.L/2) - (self.lpar+(self.bond_L / 2))) #reducing step size for random walk
        print('range',rand_range)
        for i in range(length):
            translate_by = np.random.uniform(low=-1, high=1, size=(3,))
            translate_by /= np.linalg.norm(translate_by)*self.bond_L
            this_bead = mb.clone(bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
                this_bead.translate(by=self.pbc(translate_by+last_bead.pos,box=([rand_range]*3)))
            else:
                translate_by = np.random.uniform(low=-rand_range, high=rand_range, size=(3,))
                this_bead.translate(by=self.pbc(translate_by,box=([rand_range]*3)))
            chain.add(this_bead)
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"
        return chain

    def pbc(self,d,box):
        for i in range(3):
            while d[i] > box[i]/2 or d[i] < -(box[i]/2):
                #print('using pbcs')
                if d[i] < -box[i]/2:
                    d[i] += box[i]
                if d[i] > box[i]/2:
                    d[i] -= box[i]
        return d

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=1.1,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

class RandomSystem(System):
    def __init__(self, molecules, base_units=dict()):
        self.L = molecules.L
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        comp = mb.Compound()
        comp.add(chain)
        comp.box = mb.Box(lengths=np.array([self.L] * 3))
        return comp

def check_inter_particle_distance(positions,box,minimum_distance=0.95):
    '''
    Check particle separations.
    
    '''
    aq = freud.locality.AABBQuery(box,positions)
    aq_query = aq.query(
        query_points=positions,
        query_args=dict(r_min=0.0, r_max=minimum_distance, exclude_ii=True),
    )
    nlist = aq_query.toNeighborList()
    if len(nlist)==0:
        print("Inter-particle separation reached.")
        return True
    else:
        return False

In [3]:
chains = EllipsoidChainRand(lengths=10,num_mols=10,lpar=0.5,bead_mass=1.0,density=0.9)
ff = DPD_FF(epsilon=1.0,lpar=0.5,lperp=0.5,A=5000,gamma=800,kT=2.0,r_cut=1.15,bond_k=25000,bond_r0=0.1)
ff.hoomd_forces
system = RandomSystem(molecules=chains)
system.to_gsd('test.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=0.5,lperp=0.5
)
sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

sim.save_restart_gsd()
sim.run_NVT(n_steps=1000, kT=2.0, tau_kt=10*sim.dt)
sim.flush_writers()

box = sim.box_lengths

with gsd.hoomd.open('trajectory.gsd', "r") as traj:
    snap = traj[-1]
    typids = snap.particles.typeid
    r_idx = np.where(typids == 0)
    positions = snap.particles.position[r_idx[0]]

while not check_inter_particle_distance(positions,box,minimum_distance=0.9):
    sim.run_NVT(n_steps=1000, kT=2.0, tau_kt=10*sim.dt)
    sim.flush_writers()
    with gsd.hoomd.open('trajectory.gsd', "r") as traj:
        snap = traj[-1]
        r_idx = np.where(typids == 0)
        positions = snap.particles.position[r_idx[0]]

range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
range 1.8537492838456806
Initializing simulation state from a gsd.hoomd.Frame.
Step 100 of 1000; TPS: 343.33; ETA: 0.0 minutes
Step 200 of 1000; TPS: 475.98; ETA: 0.0 minutes
Step 300 of 1000; TPS: 553.25; ETA: 0.0 minutes
Step 400 of 1000; TPS: 606.06; ETA: 0.0 minutes
Step 500 of 1000; TPS: 647.37; ETA: 0.0 minutes
Step 600 of 1000; TPS: 679.52; ETA: 0.0 minutes
Step 700 of 1000; TPS: 709.42; ETA: 0.0 minutes
Step 800 of 1000; TPS: 737.16; ETA: 0.0 minutes
Step 900 of 1000; TPS: 764.27; ETA: 0.0 minutes
Step 0 of 1000; TPS: 0.0; ETA: nan hours, nan minutes
Step 100 of 1000; TPS: 1244.54; ETA: 0.0 minutes
Step 200 of 1000; TPS: 1278.98; ETA: 0.0 minutes
Step 300 of 1000; TPS: 1319.0; ETA: 0.0 minutes
Step 400 of 1000; TPS: 1363.37; ETA: 0.0 minutes
Step 500 of 100

# Getting orientation vectors and 'T' anchor positions from trajectory

In [4]:
snap = sim.state

In [5]:
traj = gsd.hoomd.open('trajectory.gsd','r')

In [6]:
last_frame = traj[-1]

In [7]:
orientations = last_frame.particles.orientation

In [8]:
N = 16
print(last_frame.particles.typeid, last_frame.particles.types)
print("orientations of rigid bodies\n",orientations[0:N])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 2 2 3 1 2 2 3 1 2
 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2
 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1
 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2
 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2
 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1
 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2
 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2
 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 

In [9]:
#positions of T particles
positions = last_frame.particles.position
typids = last_frame.particles.typeid
print(last_frame.particles.types)
t_idx = np.where(typids == 2)

['R', 'A', 'T', 'X']


In [10]:
t_positions = positions[t_idx[0]]

In [11]:
print('ellipsoid anchor positions',t_positions)

ellipsoid anchor positions [[-7.25435853e-01  1.80737615e+00 -1.18664587e+00]
 [-1.47787988e+00  1.24115634e+00 -1.52313244e+00]
 [ 2.98479885e-01  2.09556651e+00 -1.39767802e+00]
 [-6.54814661e-01  1.88702595e+00 -1.17918229e+00]
 [ 1.22894073e+00  2.07542872e+00 -2.00700784e+00]
 [ 3.73368144e-01  2.10356355e+00 -1.49008989e+00]
 [ 2.03307295e+00  1.34729850e+00 -2.16026807e+00]
 [ 1.35277390e+00  2.07080817e+00 -2.04310560e+00]
 [ 2.03095174e+00  2.22363874e-01 -2.07800555e+00]
 [ 2.06491256e+00  1.21726191e+00 -2.17300272e+00]
 [ 1.84926176e+00 -6.98945582e-01 -1.49014616e+00]
 [ 2.00462937e+00  1.01756252e-01 -2.06871128e+00]
 [ 1.35407388e+00 -1.34303415e+00 -7.40269482e-01]
 [ 1.81756115e+00 -7.63524830e-01 -1.41060424e+00]
 [ 7.03701496e-01 -1.58002710e+00  1.00121580e-01]
 [ 1.31740940e+00 -1.43755007e+00 -6.76449656e-01]
 [ 2.50109762e-01 -1.23077798e+00  1.08957446e+00]
 [ 6.13350034e-01 -1.56239343e+00  2.18892127e-01]
 [ 4.30123001e-01 -9.42373931e-01  2.12861800e+00]
 [ 2

# Visualizing Ellipsoids
You must add particle shape information to the gsd file for Ovito to read

In [12]:
from cmeutils.gsd_utils import ellipsoid_gsd #note this is the development version of cmeutils

GSD_PATH = 'trajectory.gsd'
lpar = lperp = 0.5
ellipsoid_gsd(
    gsd_file=GSD_PATH,
    new_file="visual-ellipsoid-dpd.gsd",
    ellipsoid_types="R",
    lpar=lpar,
    lperp=lperp,
)